<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-emergent/blob/main/EMERGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EMERGENT: barcodE iMage EnhanceR usinG dEep Neural neTworks

Autoři: Aleksej Gaj, Štěpán Studenovský

---
Cílem tohoto projektu bylo vytvořit program na rozpoznávání a čtení čárových kodů (EAN13) a 2D kódů (GS1 DataMatrix), který by byl integrovatelný do chytrých zařízení používajících fotoaparát.

V současné chvíli existují různé implementace čteček čárových kódů, čtečky GS1 DataMatrix kódů jsou velmi vzácné a citlivé na kvalitu vstupního obrázku (šum, jiné objekty na fotografii, deformace, aj.).

Motivací bylo pomocí metod strojového učení a zpracování obrazu předzpracovávat vstupní fotografie (pořizované koncovým uživatelem) pro eliminaci případů, kdy klasické čtečky selžou. Vzniklý algoritmus by měl být snadno uzpůsobitelný pro případ jak čárových, tak i 2D kódů (tj. nezávislý na typu kódu a koncové čtečce).

Protože podobných NN-řešení pro klasické čárové kódy existuje celá řada, rozhodli jsme se začít právě u nich. Po vytvoření takového modelu by bylo možné porovnat jeho úspěšnost s jiným řešením. Poté by bylo možné vyzkoušet případ méně populárních GS1 DM kódů.



## Uvažované přístupy pro konstrukci modelu
Volili jsme dva přístupy řešení:
 - konstrukce a natrénování vlastního **Image2Image** modelu, kde úloha učení by bzla specifikována pouze vhodně zvolenou strukturou sítě a dostatečně bohatým datasetem
 ``` damaged_img -> NN -> clean_img -> classic_reader ```

 Tuto cestu jsme zavrhli převážně z časových důvodů

 - detekce polohy kódu na vstupním obrázku (pomocí **object detection**), manuální vyříznutí (**crop**), a následné odšumění (**denoising** - buď NN modelem, nebo klasickými metodami zpracování obraz. inf.). Zde byly dvě možné varianty:
 > detekce kódu -> vyříznutí -> denoising -> čtení

 > denoising -> detekce -> vyříznutí -> čtení



K detekci kódu jsme se snažili použít nejprve vlastní síť vytvořenou from scratch se špatnými výsledky, tak jsme zvolili finetunování modelu Yolov5s/m. Vyříznutí jsme napsali simple python skript, který vezme výsledky z yolov5 a vyřízne obrázek.

Pro odšumění pomocí NN jsme uvažovali postavit vlastní model typu UNet, ovšem narazili jsme na problém předzpracování (nesjednocená velikost vstupu a výstupu sítě).

Proto jsme se vyzkoušeli existující řešení
[https://github.com/GunhoChoi/Kind-PyTorch-Tutorial/blob/6e98e475eda6b1a449a8470384a7978462820e52/06_Autoencoder_Model_Save/Autoencoder_Model_Save.ipynb](https://github.com/GunhoChoi/Kind-PyTorch-Tutorial/blob/6e98e475eda6b1a449a8470384a7978462820e52/06_Autoencoder_Model_Save/Autoencoder_Model_Save.ipynb)

ale klasické metody se ukázali jako časově i výpočetně snažší:

[https://colab.research.google.com/drive/1AaQzDH-BF0Wh6sJkyGIOTdmfJvgw0R72?usp=sharing](https://colab.research.google.com/drive/1AaQzDH-BF0Wh6sJkyGIOTdmfJvgw0R72?usp=sharing)





#Existující čtečka kódů (`nice_barcode -> EAN number`)
Čtení kódu jsme použili již implementovanou čtečku kódu viz. [https://pypi.org/project/python-barcode/](https://pypi.org/project/python-barcode/)

ukázka (vč. čtení kódu a generování datasetu):


[https://colab.research.google.com/drive/1DNLpUr4CrEb-vmAwVUPcHp9aV0YUUsWo?usp=sharing](https://colab.research.google.com/drive/1DNLpUr4CrEb-vmAwVUPcHp9aV0YUUsWo?usp=sharing)

Výhodou této čtečky je její velmi krátký čas na rozpoznání kódu, vysoká spolehlivost (na kvalitních fotografiích), a odolnost vůči rotaci.

Čtečka zmíněných 2D kódů existuje ( [https://github.com/NaturalHistoryMuseum/pylibdmtx/](https://github.com/NaturalHistoryMuseum/pylibdmtx/) ), ovšem žádnou z těchto kvalit nemá. Čtení kódu je pomalé, často neuspěšné a extrémně citlivé na vstupní obrázek.

# Dataset

### Příprava reálného datasetu
Pro detekci čárových kódu jsme nejprve připravili datasety, některé již anotované: https://www.kaggle.com/datasets/whoosis/barcode-detection-annotated-dataset, http://artelab.dista.uninsubria.it/downloads/datasets/barcode/medium_barcode_1d/medium_barcode_1d.html,

použili jsme ještě jeden dataset ze stránky kaggle.com, kde bylo přibližně 500 obrázků anotovaných  teď již ručně pomocí https://www.makesense.ai/, výborný nastroj pro anotování obrázků pro formát YOLO.

Taktéž jsme vyvinuli program pro generování datasetů vhodných pro trénování NN na fotografiích kódů.

### Princip generování umělého datasetu:
 - podle náhodného klíče se vygenerují obrázky čárových kódů
 - následně jsou zašuměny kombinací Gauss. a Poisson. šumů
 - poté otočeny o náhodný úhel
 - zmenšeny a umístěny na jiný, předem zvolený obrázek (sadu obrázků)

Výhody:
 - snadná a rychlá generace i velkých datasetů (10min - 3000ks i víc)
 - vysoká míra nastavitelnosti parametrů a složitosti
  
Nevýhody:
 - není jisté, zda se učení na takovém datasetu bude podobat realitě (ostatně tot byla motivace používat různé datasety)





Ukázka generování datasetu
[https://colab.research.google.com/drive/1DNLpUr4CrEb-vmAwVUPcHp9aV0YUUsWo?usp=sharing](https://colab.research.google.com/drive/1DNLpUr4CrEb-vmAwVUPcHp9aV0YUUsWo?usp=sharing)

a samotného datasetu:
[https://huggingface.co/datasets/aleksejalex/barcodes_temp](https://huggingface.co/datasets/aleksejalex/barcodes_temp)

# Metodologie

Co se detekce čárových kódů a typu úlohy, bylo by nejjednodušší navrhnout nebo použít sítě typu Yolo, fast RNN.... Síť by měla detekovat čárový kód, udělat kolem něj obdélník, pojmenovat třídu, do které jej klasifikovala a poslat výsledek predikce. Taktéž by zaznamenala kde se výřez na fotce vyskytuje pro budoucí vystřihování.

Jelikož se jedná o poměrně jednoduchou úlohu detekce jen jedné třídy, tak jsme nejprve zvolili cestu pro vytvoření a natrénování vlastní neuronové sítě v jazyce pytorch-lightning. Trénovali jsme přes Helios na FJFI, dále v colab.google na T4. Vlastní síť se neukázala příliš efektivní, nejspíše by to chtělo mnohem složitější architekturu sítě, vytvořit vlastní loss funkci a hlubší porozumění problému - https://colab.research.google.com/drive/1dq53A7OU2OXf5SzzKatDl7_9VUder5bu?usp=sharing.py.

Výsledky trénování a testování jsme ukládali pomocí Weights and Biases.

Druhý pokus byl o dotrénování modelu Yolov5s, pokus byl opět neúspěšný. Snaha byla pouze načíst model a vytvořit vlastní přístup a prostředí pro dotrénování - viz AlexNet na cvikách. Nepodařilo se díky nezkušenosti, neschopnosti pracujícího a hlavně díky komplikovanosti modelu implementovat jednotlivé prvky - loss funkci. Odkaz na colab - https://colab.research.google.com/drive/1GKQUOvoa0GSPtYpWARyyaOSIZxiRiaVB?usp=sharing

Poslední pokus jsme se věnovali postupu doporučenému od Ultralytics na stránce https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#next-steps. Zde jsme již po další úpravě datových souborů a následování tutoriálu došli k výsledku v https://colab.research.google.com/drive/1Kn7DAfYJPsnvY6c9FK12aPYXOgdDtO0O?usp=sharing. Výsledky trénování se již ukázali jako úspěšné a barcode klasifikoval správně.


Vyříznutí obrázku, jsme implemetovali jednoduchý Python skript, který si bere predikovaný .txt/.json výstup z Yolov5 a ořízne obrázek dle klasifikace. https://colab.research.google.com/drive/1tx5v-YymIFhwkDBcqea0apRgVOBslwgE?usp=sharing


# Závěr
Téma této práce bylo motivováno aplikací: v současné době vznikající webová aplikace "Chytrá lékarnička" potřebovala robustní a univerzální způsob úpravy fotografií různých kódů pro následné čtení.

Ovšem kombinace faktorů, jako
 - práce na dost abstraktní úloze (zrobustnit existující nástroje)
 - kombinování několika nástrojů, které ani jeden z členů týmu neuměl ovládat a museli jsme se to naučit "za pochodu"
 - časová tíseň

způsobili, že v tuto chvíli nelze předvést demoverzi v podobě funkční pipeline. Avšak výstupy, které můžeme poskytnout, jsou:
 - autoři mají komplexní představu o doméně, v níž byla úloha formulována, a přehled o existujících postupech a jejich slabinách
 - několik .ipynb notebooků, ve kterých jsou podrobně sestrojeny a vyzkoušeny jednotlivé prvky výsledného programu, které bohužel z časových důvodů nebylo možné sjednotit
 - autoři se naučili ovládat technické postupy jako přetrénování YOLOv5, generování umělých obrazových datasetů, techniky odšumování (denoising), aj.



Neúspěšnost při sestrojování vlastní NN nebereme za neúspěch, pro úplně první vlastní implementaci NN pro netriviální problém to byl hezký pokus, na kterém jsme se hodně naučili.

Do budoucna - projekt nezaniká právě kvůli výše zmíněné motivaci. Tedy  pokračování by mohlo být:
 - vylepšit architekturu vlastní implementace a zlepšit loss funkci, přidat další parametry
 - vyzkoušet pokročilé techniky denoising jako např. noise2void ( [https://github.com/juglab/n2v](https://github.com/juglab/n2v) )
 - do generování umělých datasetů zahrnout i deformace obrazu

